# Метод Apriori ассоциативного анализа

Написанного Ю Мочизуки (Yu Mochizuki)
Устанавливаем

```
pip install apyori
copy apyori.py into your project.
python setup.py install
```


In [ ]:
import pandas as pd

# Данные https://www.kaggle.com/datasets/devchauhan1/market-basket-optimisationcsv
dataset = pd.read_csv("./data/Market_Basket_Optimisation.csv", header=None)
dataset.head()

### Очистка данных


In [ ]:
dataset.fillna(method="ffill", axis=1, inplace=True)
dataset.head()

In [ ]:
from apyori import apriori

# создадим из них матрицу
transactions = []
for i in range(0, 7501):
    transactions.append([str(dataset.values[i, j]) for j in range(0, 20)])
""" 
и обучимся правилам. Обратите внимание, что пороговые значения мы выбираем сами в зависимости от того,
насколько "сильные" правила мы хотим получить
min_support -- минимальный support для правил (dtype = float).
min_confidence -- минимальное значение confidence для правил (dtype = float)
min_lift -- минимальный lift (dtype = float)
max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer) 
"""
result = list(
    apriori(
        transactions, min_support=0.003, min_confidence=0.2, min_lift=4, min_length=2
    )
)
apriori

In [ ]:
import shutil, os
from io import StringIO
from apyori import dump_as_json
from IPython.display import display, HTML
import json

output = []
for RelationRecord in result:
    o = StringIO()
    dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
df = pd.DataFrame(output)
df.sort_values(by=["support"], ascending=False)

### Результат

Видим сочетания:

- оливковое масло, макароны из цельнозерновой муки [olive oil, whole wheat pasta],
- говяжий фарш, зелень и перец, спагетти [ground beef, herb & pepper, spaghetti],
- эскалоп, макароны [escalope, pasta]


### Теоретическая часть

- https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html
- https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/


# Эффективный вариант Apriori с использованием сокращения хэшей


In [21]:
""" 
Эффективный вариант Apriori с использованием сокращения хэшей
"""

from itertools import combinations


class hashTable:
    def __init__(self, hash_table_size):
        self.hash_table = [0] * hash_table_size

    def add_itemset(self, itemset):
        hash_index = (itemset[0] * 10 + itemset[1]) % 7
        self.hash_table[hash_index] += 1

    def get_itemset_count(self, itemset):
        hash_index = (itemset[0] * 10 + itemset[1]) % 7
        return self.hash_table[hash_index]


def generateCandidateItemsets(level_k, level_frequent_itemsets):
    """Генерация и сокращение наборов элементов-кандидатов для следующего уровня с использованием часто используемых наборов элементов текущего уровня.
    @ Параметры
    ----------
    level_k : целое число
            Текущий номер уровня

    level_frequent_itemsets : список списков
            Список частых наборов предметов текущего уровня
    @ Возвращает
    -------
    список списков
            Наборы-кандидаты следующего уровня
    """
    n_frequent_itemsets = len(level_frequent_itemsets)
    candidate_frequent_itemsets = []
    for i in range(n_frequent_itemsets):
        j = i + 1
        while (j < n_frequent_itemsets) and (
            level_frequent_itemsets[i][: level_k - 1]
            == level_frequent_itemsets[j][: level_k - 1]
        ):
            candidate_itemset = (
                level_frequent_itemsets[i][: level_k - 1]
                + [level_frequent_itemsets[i][level_k - 1]]
                + [level_frequent_itemsets[j][level_k - 1]]
            )
            candidate_itemset_pass = False

            if level_k == 1:
                candidate_itemset_pass = True

            elif (level_k == 2) and (candidate_itemset[-2:] in level_frequent_itemsets):
                candidate_itemset_pass = True

            elif all(
                (list(_) + candidate_itemset[-2:]) in level_frequent_itemsets
                for _ in combinations(candidate_itemset[:-2], level_k - 2)
            ):
                candidate_itemset_pass = True

            if candidate_itemset_pass:
                candidate_frequent_itemsets.append(candidate_itemset)

            j += 1

    return candidate_frequent_itemsets


def aprioriAlgorithm(transactions, min_support_count):
    """Извлечение часто встречающихся наборов элементов из транзакций с использованием априорного алгоритма
    @ Параметры
    ----------
    транзакции : список наборов
            Список транзакций

    min_support_count : целое число
            Минимальное количество поддержки для набора элементов, которое считается частым
    @ Возвращает
    -------
    список наборов
             Список частых наборов элементов, извлеченных из транзакций
    """

    # Извлечь список элементов в транзакциях
    items = set()
    for transaction in transactions:
        items.update(transaction)
    items = sorted(list(items))

    # Список частых наборов в транзакции
    frequent_itemsets = []

    level_k = 1  # Текущий номер уровня

    level_frequent_itemsets = []  # Level 0: частые наборы предметов
    candidate_frequent_itemsets = [
        [item] for item in items
    ]  # Level 1: наборы элементов-кандидатов

    # Инициализировать хеш-таблицу
    hash_tb = hashTable(7)

    while candidate_frequent_itemsets:
        # Подсчитаем поддержку всех возможных часто используемых наборов элементов и удалим транзакции, используя сокращение транзакций
        candidate_freq_itemsets_cnts = [0] * len(candidate_frequent_itemsets)

        for transaction in transactions:
            # добавить количество наборов элементов размера 2 в хеш-таблицу
            if level_k == 1:
                for itemset in combinations(transaction, 2):
                    hash_tb.add_itemset(itemset)

            for i, itemset in enumerate(candidate_frequent_itemsets):
                if all(_item in transaction for _item in itemset):
                    candidate_freq_itemsets_cnts[i] += 1

        # Сгенерируем часто встречающиеся наборы элементов уровня k путем сокращения нечастых наборов элементов
        level_frequent_itemsets = [
            itemset
            for itemset, support in zip(
                candidate_frequent_itemsets, candidate_freq_itemsets_cnts
            )
            if support >= min_support_count
        ]
        frequent_itemsets.extend(
            [
                set(level_frequent_itemset)
                for level_frequent_itemset in level_frequent_itemsets
            ]
        )

        # Сгенерируйте кандидатов Ck+1 из Ck (используя генерацию и обрезку)
        candidate_frequent_itemsets = generateCandidateItemsets(
            level_k, level_frequent_itemsets
        )
        level_k += 1

        # Обрежем C_2, используя хеш-таблицу, сгенерированную во время L_1
        if level_k == 2:
            for itemset in candidate_frequent_itemsets:
                if hash_tb.get_itemset_count(itemset) < min_support_count:
                    print("Pruned itemset", itemset)
                    candidate_frequent_itemsets.remove(itemset)

    return frequent_itemsets


""" Load the standard market basket dataset from datasets module """
""" 
Загрузим данные
def load_market_basket():
        import pandas as pd
        # Данные https://www.kaggle.com/datasets/devchauhan1/market-basket-optimisationcsv
        dataset = pd.read_csv('./data/market_basket.csv', header = None)
        # Preprocessing the dataset
        transactions = []
        for index, data in dataset.iterrows():
                transaction = [len(el) for el in pd.Series.tolist(data[~pd.isnull(data)])]
                transactions.append(set(transaction))
        return transactions

transactions = load_market_basket()
min_support_count = 100
verbose = False """

""" Пример данных, предложенный - Arun K. Pujari """
transactions = [
    {1, 2, 5},
    {2, 4},
    {2, 3},
    {1, 2, 4},
    {1, 3},
    {2, 3},
    {1, 3},
    {1, 2, 3, 5},
    {1, 2, 3},
    {1, 2},
    {1, 3, 5},
]

min_support_count = 3

# Сгенерируйте список всех часто используемых наборов элементов с помощью сокращения транзакций Apriori
frequent_itemsets = aprioriAlgorithm(transactions, min_support_count)

print(
    "\nЧАСТЫЕ НАБОРЫ ЭЛЕМЕНТОВ (минимальное количество поддержки = {})".format(
        min_support_count
    )
)
for frequent_itemset in frequent_itemsets:
    print(frequent_itemset)

Pruned itemset [2, 5]

ЧАСТЫЕ НАБОРЫ ЭЛЕМЕНТОВ (минимальное количество поддержки = 3)
{1}
{2}
{3}
{5}
{1, 2}
{1, 3}
{1, 5}
{2, 3}
